# An Introduction to Genify

Genify is a tool that allows users to transform stochastic code written in
plain Julia into generative functions contolled by the [Gen probabilistic
programming system](https://www.gen.dev). This tutorial show how Genify
works, and how you can use it to enable programmable inference over
stochastic Julia code.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

## Why Genify?

There are two broad use cases for Genify:
1. Programmable inference over existing stochastic Julia code (e.g. simulators)
2. PPL-compatibility for plain Julia code, without depending upon Gen
We'll motivate each of these use cases in turn.

### Programmable inference over existing stochastic code

A wide variety of libraries written in Julia implement stochastic simulators of
natural and social phenomena for the purposes of computational science. For
example, the following function implements a stochastic, discrete time,
Susceptible-Infected-Recovered (SIR) model of epidemic spread:

In [ ]:
using Random, Distributions

function sir_model(T::Int, init_pop::Vector{Int},
                   β::Float64=0.5, γ::Float64=0.02)
    # Initialize population history
    pop_history = zeros(Int, 3, T)
    pop_history[:, 1] = init_pop
    tot_pop = sum(init_pop)
    for t in 2:T
        susceptible, infected, recovered = pop_history[:, t-1]
        # Sample number of individuals who are newly infected and recovered
        newly_infected = rand(Binomial(susceptible, β * infected / tot_pop))
        newly_recovered = rand(Binomial(infected, γ))
        # Update the population counts
        susceptible -= newly_infected
        infected += newly_infected - newly_recovered
        recovered += newly_recovered
        pop_history[:, t] = [susceptible, infected, recovered]
    end
    return pop_history
end;

This allows us to perform forward simulation of how a viral epidemic
might evolve over time:

In [ ]:
using Plots
pop_history = sir_model(100, [990, 10, 0], 0.5, 0.1)
plot_sir! = (x, plt, alpha=1.0, color=palette(:default)[1:3]') ->
    plot!(plt, x', ylabel="Population", xlabel="Time", lw=2, margin=10*Plots.px,
          labels=["Susceptible" "Infected" "Recovered"], alpha=alpha,
          fg_color_legend=nothing, bg_color_legend=nothing, color=color)
plot_sir = x -> plot_sir!(x, plot(size=(800, 300), dpi=192))
plot_sir(pop_history)

However, because `sir_model` isn't written using a probabilistic
programming system like Gen, we can't perform more complex queries, such as:
- Given that 100 people were infected on day 10, how is the rest of the
  epidemic likely to evolve?
- If at most 15 people recovered every day starting from day 20, what is the
  likely recovery rate parameter, γ?

In order to answer these and other queries, we need the following capabilities
provided by a probabilistic programming system like Gen:
- Constraining of observed random variables
- Sampling random variables from custom proposal distributions
- Computing likelihoods and importance ratios for execution traces
- Seletively updating random choices and subroutines (for MCMC inference)

Here's where Genify comes to the rescue! Using the `genify` function, we can
transform `sir_model` into an equivalent generative function in Gen:

In [ ]:
using Gen, Genify
# Transform the `sir_model` method by providing its type signature
gen_sir_model = genify(sir_model, Int, Vector{Int}, Float64, Float64);

Now we can sample from this model while constraining specific random variables:

In [ ]:
# Sample a trace from the model where 50 people were infected on day 10
trace, weight = generate(gen_sir_model, (100, [990, 10, 0], 0.5, 0.1),
                         choicemap((:newly_infected => 10 - 1, 100)))
# Check that trace has the right value
@assert trace[:newly_infected => 10 - 1] == 100
# Plot sampled trace
plot_sir(get_retval(trace))

Because `gen_sir_model` supports Gen's generative function interface,
Genify allows users to perform Bayesian inference over the model using Gen's
inference library, including generic algorithms such as `importance_sampling`,
but also custom algorithms that compose Gen's inference primitives.

### PPL-compatibility for plain Julia code

Another use case for Genify is so that developers can write stochastic Julia
code that is *Gen-compatible*, but that still works without requiring Gen as a
dependency.

This might arise when implementing a randomized algorithm to solve some task
(e.g. sample-based motion planning). Most of the time, one simply uses the
algorithm to solve the task. This doesn't require a probabilistic programming
system. But sometimes, one might want to *perform inference* over the randomized
algorithm, asking questions like: what choices must the randomized algorithm
have sampled in order to produce the output it did? This sort of question
arises in domains like Bayesian inverse planning, where the task is to infer
the goals of rational agents from observations of their behavior.

From a software engineering point of view then, it would be ideal if we could
write the randomized algorithm without depending upon Gen, reducing library
bloat, but still use it with Gen when desired. Genify enables exactly this use
case, because it can transform any Julia function into a Gen function.

As a toy example, we can write a *manually addressed* Julia function that
implements an ϵ-greedy policy for a
[multi-armed bandit](https://en.wikipedia.org/wiki/Multi-armed_bandit):

In [ ]:
using TracedRandom # Provides support for rand(:addr, distribution) syntax

function bandit_policy(max_pulls::Int, levers::Vector{Float64}, ϵ::Float64)
    accum_rewards = zeros(Float64, length(levers))
    n_pulls = zeros(Int, length(levers))
    for k in 1:max_pulls
        if rand(:explore => k, Bernoulli(ϵ))
            idx = rand(:explore_idx => k, DiscreteUniform(1, length(levers)))
        else
            mean_rewards = [n > 0 ? r / n : 0.0
                            for (r, n) in zip(accum_rewards, n_pulls)]
            idx = argmax(mean_rewards)
        end
        reward = rand(:reward => k, Normal(levers[idx], 1.0))
        accum_rewards[idx] += reward
        n_pulls[idx] += 1
    end
    return (accum_rewards, n_pulls)
end;

We can then transform this using Genify's manual addressing mode. Sampling
a trace from this model shows the expected structure

In [ ]:
gen_policy = genify(:manual, bandit_policy, Int, Vector{Float64}, Float64)
trace, weight = generate(gen_policy, (5, [2.0, 5.0, 10.0], 0.1),
                         choicemap((:explore => 3, true)))
get_choices(trace)

## How does it work?

Having motivated the usefulness of Genify, we now give a brief introduction to
its implementation.

Some probabilistic programming systems, such as Pyro, make use of the fact
their host languages (e.g., Python) are interpreted languages. This enables an
implementation strategy called *non-standard interpretation*: a custom
interpreter executes code in the host language, imbuing that code with new
semantics when inference operations like conditioning are performed.

However, unlike Python, Julia is a *compiled* language, which makes it seem
difficult to implement a similar strategy. After all, if pre-written Julia
code is already compiled, how can one transform that compiled code into Gen?

### Compiler injection

Thankfully, we can overcome this challenge because Julia's just-in-time (JIT)
compilation strategy and support for meta-programming provide the following
two features:
1. Every Julia method is first lowered to a single-site assignment (SSA)
   intermediate representation (IR) before compilation to machine code.
   This IR information is stored away, and is accessible for any (non-primitive)
   Julia method.
2. Julia supports meta-programming via *generated functions*: functions with
   dynamically generated method bodies, based on the types of the provided
   arguments. This allows specialized code to be automatically generated
   and compiled when a function is called with particular types.

These two features enable an implementation strategy similar to non-standard
interpretation: *compiler injection*, also called
[*overdubbing*](https://julia.mit.edu/Cassette.jl/stable/). The basic process
is as follows:
1. Write a generated function `transform` which takes another function `fn`
   as input, along with its arguments `args`.
2. Lookup the IR of `fn` within `transform`, and modify the IR with new behavior
3. Pass the modified IR back to the Julia compiler, to get a compiled method
   `transformed_fn`, and run that method on `args`.

By modifying the IR, we can imbue the original function with new semantics:
e.g., computing likelihood weights, in addition ta sampling. Unlike non-standard
interpretation, this transformation procedure is performed only once,
during the first call to `transform(fn, args...)`. Subsequent calls to
`transform(fn, args...)` will simply execute the already compiled method body,
modifications in-hand.

### A simple example

To illustrate what this process looks like, let's see it applied to a simple
example. The following two functions define a model of noisy weighing scale,
weighing an object of unknown mass:

In [ ]:
function model(n::Int)
    mass = rand(Normal(5, 1))
    obs = observe(mass, n)
end

function observe(mass::Real, n::Int)
    obs = Float64[]
    for i in 1:n
        m = rand(Normal(mass, 2))
        push!(obs, m)
    end
    return obs
end;

Using the `IRTools.jl` package, we can inspect the IR of `model` and `observe`:

In [ ]:
using IRTools: IR, isexpr
println("== model IR ==")
IR(typeof(model), Int) |> display
println("== observe IR ==")
IR(typeof(observe), Real, Int) |> display

We can see that the IR of `model` is straightforward. The IR of `observe`
is more complicated: it has multiple basic blocks because it contains a
`for` loop.

Let's say we want to modify the behavior of `model` by replacing every
call to `rand` with some constant value. We can do this by just looping over
the IR and modifying matching statements:

In [ ]:
function derandomize_ir!(ir::IR)
    for (x, stmt) in ir
        if !isexpr(stmt.expr, :call) continue end
        fn, args = stmt.expr.args
        if !(fn isa GlobalRef && fn.name == :rand) continue end
        ir[x] = 0.0
    end
    return ir
end
IR(typeof(model), Int) |> derandomize_ir! |> display

Notice that the call to `rand` has been replaced by `0.0`.

Using `IRTools.jl`, we can perform this IR transformation within a generated
function called a *dynamo* (a "dynamic macro"):

In [ ]:
using IRTools: @dynamo
@dynamo function derandomize(args...)
    ir = IR(args...)
    return derandomize_ir!(ir)
end;

We can now call this dynamo on the function we want to derandomize:

In [ ]:
println("Derandomized `observe`:")
derandomize(observe, 10.0, 5) |> display
println("Derandomized `model`:")
derandomize(model, 5) |> display

Notice that when `observe` is derandomized, every call to `rand` is replaced
by a zero, so it ends up returning a vector of five zeroes. However, the
derandomized version of model still returns what looks like a list of random
numbers. This is because `derandomize_ir!` does not recurse into nested
functions. As result, it replaces only the call to `rand` within `model`
itself, but not within its nested call to `observe`.

To fix this, we need to modify the `derandomize` dynamo to recursively wrap
itself around nested functions:

In [ ]:
@dynamo function derandomize(args...)
    ir = IR(args...)
    if ir == nothing return end
    for (x, stmt) in ir
        if !isexpr(stmt.expr, :call) continue end
        fn, args = stmt.expr.args[1], stmt.expr.args[2:end]
        if fn isa GlobalRef && fn.name == :rand
            ir[x] = 0.0
        else
            ir[x] = Expr(:call, derandomize, fn, args...)
        end
    end
    return ir
end

println("Derandomized `model`:")
derandomize(model, 5) |> display

The derandomized version of `model` now returns a vector of five zeroes,
as desired.

This basic implementation strategy allows us to transform Julia methods with
random primitives like `rand`, `randn` or `randperm` into `DynamicDSLFunction`s
in Gen. Calls to primitives are replaced by calls to Gen distributions, wrapped
within the `Gen.traceat` method.
[Dispatch on function types](https://fluxml.ai/IRTools.jl/latest/dynamo/#Using-Dispatch-1)
simplifies the implementation considerably, and additional tricks are used to
wrap nested function calls within `DynamicDSLFunction`s. Interested readers
are encouraged to browse the source code.

## Programmable inference with Genify

Now that we've seen how Genify works, let's use it do some inference!
Recall the first query we wanted to ask about our SIR epidemic model:
- Given that 100 people were infected on day 10, how is the rest of the
  epidemic likely to evolve?

To answer this, we can just apply importance sampling while constraining the
number of newly infected to be 100 on day 10. For comparison, we also sample
simulations where only 30 people are newly infected on that day.

In [ ]:
n_samples = 50
traces_100, weights_100, _ =
    importance_sampling(gen_sir_model, (100, [990, 10, 0], 0.5, 0.1),
                        choicemap((:newly_infected => 10 - 1, 100)), n_samples)
traces_30, weights_30, _ =
    importance_sampling(gen_sir_model, (100, [990, 10, 0], 0.5, 0.1),
                        choicemap((:newly_infected => 10 - 1, 30)), n_samples);

We can then visualize the sampled traces, emphasizing those traces with
larger importance weights:

In [ ]:
plot_sir_traces = (traces, weights) -> begin
    plt = plot(size=(600, 200), dpi=192)
    for (tr, w) in zip(traces, weights)
        plot_sir!(get_retval(tr), plt, exp(w))
    end
    plot!(plt, legend=false)
end
plt_100 = plot_sir_traces(traces_100, weights_100)
title!("(a) 100 people infected on day 10")
plt_30 = plot_sir_traces(traces_30, weights_30)
title!("(b) 30 people infected on day 10")
plot(plt_100, plt_30, layout=(2, 1), size=(800, 600), dpi=192)

We can see that in simulations where 100 people are infected on day 10,
the peak in infected individuals arrives considerably earlier than when
30 people are infected on the same day.

Now let's try tackling the second query:
- If at most 15 people recovered every day starting from day 20, what is
  the likely recovery rate parameter, γ?

This query is more complex than the first, for the following reasons:
- It requires us to extend the model with uncertainty over the parameters
  β and γ, in order to perform *Bayesian parameter estimation*.
- We need some way to sample simulations where at most 15 infected individuals
  recover everyday, from day 20 onward.

Fortunately, both of these are easy to do with Gen! To extend `sir_model` with
parameter uncertainty, we can simply wrap it in an `@gen` function which
introduces priors over β and γ:

In [ ]:
@gen function param_sir_model(T::Int, init_pop::Vector{Int})
    β ~ uniform(0.0, 1.0)
    γ ~ uniform(0.0, 1.0)
    {*} ~ gen_sir_model(T, init_pop, β, γ)
end;

Notice how, in the final line, we sampled from `gen_sir_model`, the transformed
version of `sir_model`, since `gen_sir_model` is like any generative function
in Gen.

In order to sample simulations where at most 15 infected individuals recover
from day 20 onwards, the simplest thing we could do is rejection sampling:
Run `param_sir_model` many times, and throw away all samples which do not meet
the above condition. Evidently, this is wasteful, and can lead to high variance
when the condition is rare. Instead, we can use a *proposal distribution*,
specified as a generative function, that only samples simulations that meet
the condition. Furthermore, since we have Genify at our disposal, we can write
this proposal using plain Julia:

In [ ]:
function sir_proposal(T::Int, init_pop::Vector{Int}, day::Int, thresh::Int)
    # Sample latent parameters
    β = rand(Uniform(0.0, 1.0))
    γ = rand(Uniform(0.0, 1.0))
    # Initialize population history
    pop_history = zeros(Int, 3, T)
    pop_history[:, 1] = init_pop
    tot_pop = sum(init_pop)
    for t in 2:T
        susceptible, infected, recovered = pop_history[:, t-1]
        # Sample newly infected individuals
        newly_infected = rand(Binomial(susceptible, β * infected / tot_pop))
        # Sample newly recovered individuals from a truncated distribution
        r_dist = t >= day ?
            truncated(Binomial(infected, γ), -1, thresh) : Binomial(infected, γ)
        newly_recovered = rand(r_dist)
        # Update the population counts
        susceptible -= newly_infected
        infected += newly_infected - newly_recovered
        recovered += newly_recovered
        pop_history[:, t] = [susceptible, infected, recovered]
    end
    return pop_history
end
gen_sir_proposal = genify(sir_proposal, Int, Vector{Int}, Int, Int);

Notice that `sir_proposal` is almost identical to `sir_model`, except we have
made sure to (i) sample the latent parameters β and γ; (ii) sample newly
recovered individuals from a Binomial distribution truncated at the threshold
of 15. Change (i) is necessary to ensure that `sir_proposal` matches the
trace structure of `param_sir_model`, while change (ii) ensures that our
condition is always met.

Using this proposal distribution, we can now perform importance sampling
in order to estimate the posterior distribution over γ:

In [ ]:
model_args = (100, [990, 10, 0])
recov_day, recov_thresh = 20, 15
prop_args = (model_args..., recov_day, recov_thresh)
n_samples = 5000
traces, weights, _ =
    importance_sampling(param_sir_model, model_args, choicemap(),
                        gen_sir_proposal, prop_args, n_samples)

γ_mean = sum(getindex.(traces, :γ) .* exp.(weights))
println("γ posterior mean estimate: $γ_mean")
histogram(getindex.(traces, :γ), bins=20, weights=exp.(weights);
          linecolor=palette(:default)[1], legend=false,
          xlabel="γ (recovery rate)", ylabel="Probability")

Importance sampling produces a rather unintuitive posterior over the
recovery rate γ. Most of the posterior mass is towards the right, suggesting
a high recovery rate, with only a small peak for very low values of γ. This is
despite the condition stipulating that not many people recover everyday after
day 20. The intuitive explanation for this is a low recovery rate, which is
captured by the small peak to the left. However, there is another explanation:
the population recovers so quickly that by day 20, there are no more
infected individuals, and so the number of new recoveries each day is zero from
then on! Without a Bayesian analysis, we may not have discovered this competing
diversity of explanations.

That brings us to the end of this tutorial! For more examples of Genify being
used for programmable inference, check out the `examples` directory in the
[Genify.jl](https://github.com/probcomp/Genify.jl) repository. To learn more
about Gen and programmable inference, check out the website at
<https://www.gen.dev/>.